In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

# Paths to train and test directories
train_dir = './cifar-10-img/train'
test_dir = './cifar-10-img/test'

# a. Load images with Keras from directory
train_dataset = image_dataset_from_directory(
    train_dir,
    label_mode='int',
    image_size=(224, 224),  # Resizing to 224x224 for VGG16
    batch_size=32,
)

test_dataset = image_dataset_from_directory(
    test_dir,
    label_mode='int',
    image_size=(224, 224),  # Resizing to 224x224 for VGG16
    batch_size=32,
)

# Normalize the images
def normalize_images(image, label):
    return image / 255.0, label

train_dataset = train_dataset.map(normalize_images)
test_dataset = test_dataset.map(normalize_images)

Found 40079 files belonging to 10 classes.
Found 9921 files belonging to 10 classes.


In [12]:
base_model = VGG16(weights="./vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape=(224, 224, 3))

In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)

In [15]:
model = Model(inputs=base_model.input, outputs=output)

In [16]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10


  68/1253 [>.............................] - ETA: 1:39:37 - loss: 2.8102 - accuracy: 0.1365